# Gradient Boosting Desicion Tree

VER LOS VIDEOS DE STATQUEST

En las clases anteriores, observamos cómo las mejoras en los algoritmos y las optimizaciones pueden generar avances significativos en la ganancia. Ya hemos logrado un progreso considerable con los modelos de Random Forest. Hoy, daremos un paso aún más grande al explorar los modelos que actualmente están obteniendo los mejores resultados en este tipo de dominios.

## La idea de boosting
Los registros que producen error reciben un peso mayor para que se ponga más atención en resolverlos, y hecho en forma cíclica.

## La diea de gradient_boosting
Es para targets con 0 y 1 a clasificar
Arranco con todas con un mismo peso (p.ej. 0.5). O se parte de la proporción del target.
Acá se trata de predecir la diferencia entre el target y la clase/peso que se le asignó, o sea el error. Para el siguiente árbol se le suman los valores predichos y se vuelve a calcular errores y predecirlos.


Antes que nada, carguemos el entorno de trabajo


# Preparar

In [29]:
## Conectar a drive

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Librerías

In [31]:
#%pip install scikit-learn==1.3.2
#%pip install seaborn==0.13.1
#%pip install numpy==1.26.4
#%pip install matplotlib==3.7.1
#%pip install pandas==2.1.4
#%pip install lightgbm==4.4.0
%pip install optuna==3.6.1

In [32]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

import pickle

## Leer datos

In [33]:
base_path =  "/content/drive/MyDrive/Data Science y similares/Maestría Data Mining Exactas/dmeyf/dmeyf2024/"
#trabajo_path=base_path + 'TareasHogar/tp1/'
dataset_file = 'competencia_01_aumentada.csv'
#dataset_file='competencia_01_reducida.csv'
#dataset_file = 'competencia_01.csv'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'

ganancia_acierto = 273000
costo_estimulo = 7000

mes_train_desde = 202104
mes_train_hasta = 202104
mes_test = 202106

# agregue sus semillas
semillas = [111103,  111109, 111119, 112211, 111217]
semilla=semillas[2]


In [ ]:
data = pd.read_csv(dataset_path + dataset_file)
if 'clase_ternaria_num' in data.columns:
  data = data.drop(columns=['clase_ternaria_num'])

Vamos a asignar pesos a las clases. En unos minutos explicaremos las razones detrás de esta decisión. Mientras tanto, pueden aprovechar el código para ajustar el peso de la clase **BAJA+2** según lo deseen.


## Recodificar clase ternaria

In [ ]:
data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [ ]:
data['clase_binaria1'] = 0
data['clase_binaria2'] = 0
data['clase_binaria1'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)
data['clase_binaria2'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)

Y trabajaremos como es habitual en las últimas clases, con **Febrero** para entrenar y **Abril** para medir, con el fin de realizar *backtesting*

## Preparar train y test

In [ ]:
train_data = data[(data['foto_mes'] >= mes_train_desde)&(data['foto_mes'] <= mes_train_hasta)]
test_data = data[data['foto_mes'] == mes_test]

X_train = train_data.drop(['clase_ternaria','clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
y_train_binaria1 = train_data['clase_binaria1']
y_train_binaria2 = train_data['clase_binaria2']
w_train = train_data['clase_peso']

X_test = test_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
y_test_binaria1 = test_data['clase_binaria1']
y_test_class = test_data['clase_ternaria']
w_test = test_data['clase_peso']

Y preparamos el *dataset* para poder usar el **rf** de una clase anterior.

Comenzaremos explicando el funcionamiento del protagonista de esta clase: **LightGBM**. Primero, partiremos con una revisión de cómo funciona el algoritmo en el que se basa, **XGBoost**. Para una introducción completa, puedes consultar este

https://xgboost.readthedocs.io/en/stable/tutorials/model.html.

Aunque en la cátedra no somos grandes seguidores de Josh Starmer y su canal *StatQuest*, reconozco que sus series sobre *Gradient Boosting* y *XGBoost* son excelentes recursos. Aquí te dejamos los enlaces a esas dos series que realmente valen la pena:

[Serie Gradient Boosting](https://www.youtube.com/watch?v=3CC4N4z3GJc&list=PLblh5JKOoLUJjeXUvUE0maghNuY2_5fY6)

[Serie XGBoost](https://www.youtube.com/watch?v=OtD8wVaFm6E&list=PLblh5JKOoLULU0irPgs1SnKO6wqVjKUsQ)

Finalmente, analizaremos las diferencias clave que ofrece **LightGBM** frente a XGBoost. Puedes explorar más sobre ello en este https://lightgbm.readthedocs.io/en/stable/Features.html.

No olvides tener a mano la [documentación de LightGBM](https://lightgbm.readthedocs.io/)y la [lista completa de sus parámetros](https://lightgbm.readthedocs.io/en/latest/Parameters.html).

Este es un algoritmo muy usado en el mercado, recomiendo dedicarle el tiempo necesario para aprenderlo bien.

Vamos a utilizar el algoritmo directamente, sin pasar por *scikit-learn*. Sin embargo, si algún alumno lo prefiere, puede optar por usar el *wrapper* de sklearn para este caso.

Para evaluar la calidad del modelo, crearemos nuestra propia función de evaluación que calcule la ganancia. La razón de incluir los pesos es precisamente para poder implementar esta función de evaluación de manera adecuada. Al combinar las clases *BAJA+1* y *BAJA+2* en una sola, necesitamos una forma de diferenciarlas, y es aquí donde entra en juego el *weight*. Este parámetro nos permitirá distinguir entre ambas clases al momento de evaluarlas dentro del algoritmo.


# Optimizar

 ## cáclulo de ganancias

In [ ]:
# La función de evaluacion para Optuna SIEMPRE debe tener estos dos parámetros:
#  los datos predichos y los datos sobre los que se predijo, porque esto es lo que le manda
#  optuna
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    # Diferencia si eran BAJA+1 o BAJA+2
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    #Ordena ganancia según los índices ordenados de y_pred de mayor a menor
    ganancia = ganancia[np.argsort(y_pred)[::-1]] #: desde todo : hasta todo :-1 step hacia atrás
    # Ganancias acumuladas so far
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True



LGBM necesita su propio tipo de Datasets:

In [ ]:
train_data1 = lgb.Dataset(X_train, label=y_train_binaria1, weight=w_train)
train_data2 = lgb.Dataset(X_train, label=y_train_binaria2, weight=w_train)

## Función objetivo para optuna

In [ ]:
def objective(trial):

    num_leaves = trial.suggest_int('num_leaves', 8, 300),
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.3), # mas bajo, más iteraciones necesita
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 1500),
    feature_fraction = trial.suggest_float('feature_fraction', 0.1, 1.0),
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.1, 1.0),

    # Hiperparámetros adicionales
    #lambda_l1 = trial.suggest_float('lambda_l1', 0.0, 10.0)
    #lambda_l2 = trial.suggest_float('lambda_l2', 0.0, 10.0)
    min_gain_to_split = trial.suggest_float('min_gain_to_split', 0.0, 1.0)
    #max_depth = trial.suggest_int('max_depth', -1, 50)
    #bagging_freq = trial.suggest_int('bagging_freq', 0, 10)
    #colsample_bytree = trial.suggest_float('colsample_bytree', 0.1, 1.0)
    #early_stopping_rounds = trial.suggest_int('early_stopping_rounds', 10, 100)

    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'seed': semilla,
        'verbose': -1,

        # Hiperparámetros agregados
        'min_gain_to_split': min_gain_to_split,
        #'max_depth': max_depth,
        #'lambda_l1': lambda_l1,
        #'lambda_l2': lambda_l2,
        #'bagging_freq': bagging_freq,
        #'colsample_bytree': colsample_bytree,
        #'early_stopping_rounds': early_stopping_rounds,

    }
    train_data = lgb.Dataset(X_train,
                              label=y_train_binaria2, # eligir la clase
                              weight=w_train)
    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=200, # modificar, subit y subir... y descomentar la línea inferior. Subirlo muchísimo.
        #early_stopping_rounds= int(50 + 5 / learning_rate),         # Si subimos mucho el num_boost_round (más árboles), podemos hacer que termine por aquí. Es cant de rondas sin superar
        # aquí la función sabe por sí misma que tiene que usar como parámetros las predicciones y los datos
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,              #no lo muestra en el log pero lo está haciendo
        seed=semillas[1]
    )
    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5        # Son 5 k-folds... parece que es por eso.


## Optuna

In [ ]:
storage_name = "sqlite:///" + db_path + "optimization_lgbm.db"
study_name = "lgbm_muy_ampliada"

# Crea el estudio

# Si quiero recomenzar, desmarcar la siguiente línea
#optuna.delete_study(study_name=study_name, storage=storage_name)
study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

In [ ]:
#study.optimize(objective, n_trials=1) # subir subir
study.optimize(objective, n_trials=300) # subir subir

Analizamos los resultados as usual

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
plot_param_importances(study)

El **learning rate** es un parámetro que tiene que ir acompañado por más árboles.

In [ ]:
plot_slice(study)

In [ ]:
plot_contour(study)

In [ ]:
plot_contour(study, params=['num_leaves','min_data_in_leaf'] )

Y finalmente tomamos el mejor modelo y lo entrenamos con la totalidad de los datos

In [ ]:
best_iter = study.best_trial.user_attrs["best_iter"]
print(f"Mejor cantidad de árboles para el mejor model {best_iter}")

In [ ]:
best_iter = study.best_trial.user_attrs["best_iter"]
print(f"Mejor cantidad de árboles para el mejor model {best_iter}")
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': study.best_trial.params['num_leaves'],
    'learning_rate': study.best_trial.params['learning_rate'],
    'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
    'feature_fraction': study.best_trial.params['feature_fraction'],
    'bagging_fraction': study.best_trial.params['bagging_fraction'],
    'min_gain_to_split': study.best_trial.params['min_gain_to_split'],
 #   'early_stopping_rounds': study.best_trial.params['early_stopping_rounds'],

    'seed': semilla,
    'verbose': 0
}

train_data = lgb.Dataset(X_train,
                          label=y_train_binaria2,
                          weight=w_train)

model = lgb.train(params,
                  train_data,
                  num_boost_round=best_iter)


Observamos la variables más importantes para el modelo:

In [ ]:
lgb.plot_importance(model, figsize=(10, 20))
plt.show()


Y si queremos tener las variables más importantes en forma de *Dataframe*:

In [ ]:
importances = model.feature_importance()
feature_names = X_train.columns.tolist()
importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
importance_df = importance_df.sort_values('importance', ascending=False)
importance_df[importance_df['importance'] > 0]


Para guardar el modelo para poder utilizarlo más adelante, no es necesario guardarlo como *pickle*, la librería nos permite guardarlo en formato texto

In [ ]:
model.save_model(modelos_path + 'lgb_muy_ampliada_abril.txt')

Y recuperar el mismo desde ese formato

In [ ]:
model = lgb.Booster(model_file=modelos_path + 'lgb_muy_ampliada_abril.txt')

Ahora vamos a entrenar en **Abril  marzo** y predecir en  **Junio**

> Añadir blockquote



# Entrenar en Abril

In [ ]:
train_data = lgb.Dataset(X_train,
                          label=y_train_binaria2,
                          weight=w_train)

model = lgb.train(params,
                  train_data,
                  num_boost_round=best_iter)


## Predecir en junio

In [ ]:
y_pred=model.predict(X_test)

## Tarea:

1. **Generar Dataset**  
   - Utilice las técnicas de *feature engineering* vistas en las clases anteriores para generar un nuevo conjunto de datos.
   
2. **Optimización de LightGBM (LGBM)**  
   - Ajuste el modelo de LightGBM utilizando una mayor cantidad de árboles y realice una exploración más exhaustiva de los hiperparámetros para mejorar su rendimiento.
   
3. **Incluir Nuevos Parámetros en la Optimización**  
   - Revise la documentación de los parámetros de LightGBM. Evalúe la inclusión de otros parámetros en el proceso de optimización, y ajuste el modelo con estos nuevos parámetros.
   
4. **Selección del Mejor Modelo**  
   - Entre los cinco mejores modelos obtenidos en cada optimización, seleccione el que considere más adecuado para la competencia en Kaggle.
   - Documente las pruebas que realizó para seleccionar el mejor modelo. Justifique su decisión con métricas relevantes y análisis comparativos.
5. Escriba y comparta por **Zulip** una función que envíe prepare el dataset que es necesario enviar a **kaggle** con los N clientes con mayor probabilidad.

## Enviar a Kaggle



In [ ]:
# Cargo el kaggle.json para poder conectarme
import os
import shutil

# Mueve kaggle.json al directorio correcto
os.makedirs('/root/.kaggle', exist_ok=True)
kaggle_json_path="/content/drive/MyDrive/Data Science y similares/Maestría Data Mining Exactas"
shutil.copy(f'{kaggle_json_path}/kaggle.json', '/root/.kaggle/kaggle.json')
os.chmod('/root/.kaggle/kaggle.json', 600)



In [ ]:
  # En y_pred está la predicción hecha sobre junio
  # En X_test los datos de junio sobre los que se hizo la predicción

  # Obtengo el orden de prioridad para el envío
idx_deseables=np.argsort(y_pred)[::-1]


  # --------------- fijo los parámetros de envío ---------------------
min_envios=8500
max_envios=16000
cant_envios=10
  # ------------------------------------------------------------------

for envios in np.linspace(min_envios, max_envios, cant_envios, dtype=int):
    print (f"Enviando: {envios}")
    # Notar que idx_deseables tiene índices de y_pred (de 0 al máximo de línes de y_pred)

    # los primeros envíos irán con predicción 1 y el resto 0
    # idx_deseables contiene los índices de y_pred (no los originales!!) cuyas predicciones tienen los mejores valores, en orden desdendente
    # guardo en elegidos los índices de los datos correspondientes a los primeros :envios en idx
    elegidos=X_test.index[idx_deseables[:envios]]

    # Creo clientes_kaggle sólo con el número de cliente de  X_test
    clientes_kaggle=X_test.loc[:, ['numero_de_cliente']].copy()
    clientes_kaggle['Predicted']=0    #default
    clientes_kaggle.loc[elegidos, 'Predicted'] = 1   # marco los "enviados"

    if clientes_kaggle.shape != (164876, 2):
        raise ValueError(f"El DataFrame 'clientes_enviados' no tiene la forma esperada. Forma actual: {clientes_elegidos.shape}")
    else:
        print("La forma de 'clientes_enviados' es correcta.")

    # elijo el nombre para el archivo enviar, para llevar control
    # un mensaje para registrar
    # y me grabo el archivo que mando para tener control local
    version=31
    # Voy a mandar con un mensaje que incluya los parámetros usados y la cantidad de envíos
    mensaje=f"lgbm muy aumentada semilla {semilla} {params} entrenado abril-marzo pred junio envios {envios}"
    # Me gusta guardar los envíos que hago
    archivo=f"k601_{str(version).zfill(3)}-{envios}.csv"
    path_archivo=dataset_path+archivo
    clientes_kaggle.to_csv(path_archivo, index=False)

    # Envío a la competencia
    competencia="dm-ey-f-2024-primera"
    !kaggle competitions submit -c {competencia} -f '{path_archivo}' -m '{mensaje}'




In [ ]:
params_str = str(params)

# Eliminar llaves {}, apóstrofes ' y dos puntos :
params_str = params_str.replace('{', '').replace('}', '').replace("'", '').replace(':', '').replace(",",'')
params_str

In [ ]:
clientes_kaggle